In [12]:
import json
import os
import re
from pathlib import Path

import pandas as pd
import numpy as np
import omero.clients

from omero.gateway import BlitzGateway

In [13]:
output_dir = '/diskmnt/Projects/Users/estorrs/imaging-analysis/results/brca_subtype/revision_v1'
Path(output_dir).mkdir(parents=True, exist_ok=True)

In [14]:
password = ''

In [15]:
group = 'HTAN'
project = 'CODEX_initial'
username = 'estorrs'
host = 'htan-imaging.wucon.wustl.edu'
port = '4064'

In [16]:
conn = BlitzGateway(username, password, host=host, port=port,
                    secure=True, group=group)
conn.connect()

True

In [17]:
project = next(iter(conn.getObjects("Project", attributes={"name": project})))

In [18]:
# list out all images in the project
name_to_img = {}
for dataset in project.listChildren():
    for img in dataset.listChildren():
        name_to_img[f'{dataset.getName()} | {img.getName()}'] = img
sorted(name_to_img.keys())

['E16.5_female_section1_04272023 | E16.5_female_section1_04272023',
 'E16.5_female_section2_04272023 | E16.5_female_section2_04272023',
 'E16.5_male_section1_04272023 | E16.5_male_section1_04272023',
 'E16.5_male_section2_04272023 | E16.5_male_section2_04272023',
 'HS-18-11250-1B_06102023 | HS-18-11250-1B_06102023.ome.tiff',
 'HS-18-6934-2A_06062023 | HS-18-6934-2A_06062023.ome.tiff',
 'HS-19-14816-1J_06062023 | HS-19-14816-1J_06062023.ome.tiff',
 'HS-19-6647-1B_06102023 | HS-19-6647-1B_06102023.ome.tiff',
 'HT110B1_S1H4 | HT110B1_S1H4',
 'HT112C1 | HT112C1',
 'HT171B1-S1H9A1-4_04192023 | HT171B1-S1H9A1-4_04192023',
 'HT171B1-S1H9A1-4_left_05122023 | HT171B1-S1H9A1-4_left_05122023.ome.tiff',
 'HT171B1-S1H9A1-4_right_05122023 | HT171B1-S1H9A1-4_right_05122023.ome.tiff',
 'HT171B1-S1H9A1-A4_left_05262023 | HT171B1-S1H9A1-A4_left_05262023.ome.tiff',
 'HT171B1-S1H9A1-A4_right_05262023 | HT171B1-S1H9A1-A4_right_05262023.ome.tiff',
 'HT171B1_S1H1A1_11182022 | HT171B1_S1H1A1_11182022',
 'HT17

In [19]:
names = [
    'HT110B1_S1H4 | HT110B1_S1H4',
    'HT171B1-S1H9A1-4_04192023 | HT171B1-S1H9A1-4_04192023',
    'HT171B1-S1H9A1-4_left_05122023 | HT171B1-S1H9A1-4_left_05122023.ome.tiff',
    'HT171B1-S1H9A1-4_right_05122023 | HT171B1-S1H9A1-4_right_05122023.ome.tiff',
    'HT171B1-S1H9A1-A4_left_05262023 | HT171B1-S1H9A1-A4_left_05262023.ome.tiff',
    'HT171B1-S1H9A1-A4_right_05262023 | HT171B1-S1H9A1-A4_right_05262023.ome.tiff',
    'HT206B1-S1H5_U1_07012023 | HT206B1-S1H5_U1_07012023.ome.tiff',
    'HT206B1-S1H5_U2_07012023 | HT206B1-S1H5_U2_07012023.ome.tiff',
    'HT206B1_H1 | HT206B1_H1',
    'HT206B1_H1_06252022 | HT206B1_H1_06252022',
    'HT206B1_S1H2L4_20221020 | HT206B1_S1H2L4_20221020',
    'HT206B1_S1H2L4_20221028 | HT206B1_S1H2L4_20221028',
    'HT243B1-S1H4A4_04192023 | HT243B1-S1H4A4_04192023',
    'HT243B1-S1H4A4_left_05122023 | HT243B1-S1H4A4_left_05122023.ome.tiff',
    'HT243B1-S1H4A4_left_05262023 | HT243B1-S1H4A4_left_05262023.ome.tiff',
    'HT243B1-S1H4A4_right_05122023 | HT243B1-S1H4A4_right_05122023.ome.tiff',
    'HT243B1-S1H4A4_right_05262023 | HT243B1-S1H4A4_right_05262023.ome.tiff',
    'HT243B1_S1H4 | HT243B1_S1H4',
    'HT271B1-S1H6A5_04192023 | HT271B1-S1H6A5_04192023',
    'HT271B1-S1H6A5_left_05122023 | HT271B1-S1H6A5_left_05122023.ome.tiff',
    'HT271B1-S1H6A5_left_05262023 | HT271B1-S1H6A5_left_05262023.ome.tiff',
    'HT271B1-S1H6A5_mid_05262023 | HT271B1-S1H6A5_mid_05262023.ome.tiff',
    'HT271B1-S1H6A5_right_05122023 | HT271B1-S1H6A5_right_05122023.ome.tiff',
    'HT271B1-S1H6A5_right_05262023 | HT271B1-S1H6A5_right_05262023.ome.tiff',
    'HT271B1_S1H3A5_02172023 | HT271B1_S1H3A5_02172023',
    'HT297B1-S1H1_U1_07012023 | HT297B1-S1H1_U1_07012023.ome.tiff',
    'HT297B1-S1H1_U2_07012023 | HT297B1-S1H1_U2_07012023.ome.tiff',
    'HT297B1_H1_08042022 | HT297B1_H1_08042022',
    'HT305B1_S1H1 | HT305B1_S1H1',
    'HT308B1-S1H5A4_04192023 | HT308B1-S1H5A4_04192023',
    'HT308B1-S1H5A4_left_05122023 | HT308B1-S1H5A4_left_05122023.ome.tiff',
    'HT308B1-S1H5A4_left_05262023 | HT308B1-S1H5A4_left_05262023.ome.tiff',
    'HT308B1-S1H5A4_right_05122023 | HT308B1-S1H5A4_right_05122023.ome.tiff',
    'HT308B1-S1H5A4_right_05262023 | HT308B1-S1H5A4_right_05262023.ome.tiff',
    'HT323B1_H1A1 | HT323B1_H1A1',
    'HT323B1_H1A4 | HT323B1_H1A4',
    'HT323B1_H1_08042022 | HT323B1_H1_08042022',
    'HT323B1_H3 | HT323B1_H3',
    'HT339B1_H1A1 | HT339B1_H1A1',
    'HT339B1_H2A1 | HT339B1_H2A1',
    'HT339B1_H4A4 | HT339B1_H4A4',
    'HT339B2-H1_04262023 | HT339B2-H1_04262023',
    'HT339B2-H2_U1_07012023 | HT339B2-H2_U1_07012023.ome.tiff',
    'HT339B2-H2_U2_07012023 | HT339B2-H2_U2_07012023.ome.tiff',
    'HT365B1_S1H1_02132023 | HT365B1_S1H1_02132023',
    'HT397B1_H2A2 | HT397B1_H2A2',
    'HT397B1_H3A1 | HT397B1_H3A1',
    'HT397B1_S1H1A3U22_04122023 | HT397B1_S1H1A3U22_04122023',
    'HT397B1_S1H1A3U31_04062023 | HT397B1_S1H1A3U31_04062023',
    'HT397B1_U12_03172023 | HT397B1_U12_03172023',
    'HT397B1_U2_03162023 | HT397B1_U2_03162023',
    'HT480B1_S1H2_R001 | HT480B1_S1H2_R001',
    'HT480B1_S1H2_R002 | HT480B1_S1H2_R002',
    'HT565B1-H2_04262023 | HT565B1-H2_04262023',
    'HT565B1-S1H2_U1_07012023 | HT565B1-S1H2_U1_07012023.ome.tiff',
    'HT565B1-S1H2_U2_07012023 | HT565B1-S1H2_U2_07012023.ome.tiff',
]

In [20]:
channel_thresholds = {}
for name in names:
    img = name_to_img[name]
    rendering_defs = img.getAllRenderingDefs()
    channel_metadata = rendering_defs[0]['c']
    thresholds = [c['start'] for c in channel_metadata]
    
    if np.asarray(thresholds).sum() == 0:
        print('warning: all thresholds are zero. maybe they are not set for this image?', name)

    channel_thresholds[name] = thresholds
print('{')
for k in channel_thresholds.keys():
    print(f"'{k}': ")
print('}')
# set(channel_thresholds.keys())

{
'HT110B1_S1H4 | HT110B1_S1H4': 
'HT171B1-S1H9A1-4_04192023 | HT171B1-S1H9A1-4_04192023': 
'HT171B1-S1H9A1-4_left_05122023 | HT171B1-S1H9A1-4_left_05122023.ome.tiff': 
'HT171B1-S1H9A1-4_right_05122023 | HT171B1-S1H9A1-4_right_05122023.ome.tiff': 
'HT171B1-S1H9A1-A4_left_05262023 | HT171B1-S1H9A1-A4_left_05262023.ome.tiff': 
'HT171B1-S1H9A1-A4_right_05262023 | HT171B1-S1H9A1-A4_right_05262023.ome.tiff': 
'HT206B1-S1H5_U1_07012023 | HT206B1-S1H5_U1_07012023.ome.tiff': 
'HT206B1-S1H5_U2_07012023 | HT206B1-S1H5_U2_07012023.ome.tiff': 
'HT206B1_H1 | HT206B1_H1': 
'HT206B1_H1_06252022 | HT206B1_H1_06252022': 
'HT206B1_S1H2L4_20221020 | HT206B1_S1H2L4_20221020': 
'HT206B1_S1H2L4_20221028 | HT206B1_S1H2L4_20221028': 
'HT243B1-S1H4A4_04192023 | HT243B1-S1H4A4_04192023': 
'HT243B1-S1H4A4_left_05122023 | HT243B1-S1H4A4_left_05122023.ome.tiff': 
'HT243B1-S1H4A4_left_05262023 | HT243B1-S1H4A4_left_05262023.ome.tiff': 
'HT243B1-S1H4A4_right_05122023 | HT243B1-S1H4A4_right_05122023.ome.tiff': 
'HT24

In [21]:
json.dump(channel_thresholds, open(os.path.join(output_dir, 'channel_thresholds.json'), 'w'))

In [22]:
conn.close()